# 의사록 데이터 크롤링

In [ ]:
pip install tika

In [51]:
# 의사록 Crawling

import requests
from bs4 import BeautifulSoup
from tika import parser
import pandas as pd

start_page = 1
current_page = start_page
end_page = 31

# 32페이지는 링크따와서 수작업
host = 'http://www.bok.or.kr/'
result_df = pd.DataFrame(columns=['date', 'text'])

while True:

    URL = 'http://www.bok.or.kr/portal/bbs/B0000245/list.do?menuNo=200761&pageIndex={}'.format(current_page)
    r = requests.get(URL)
    sp = BeautifulSoup(r.text, 'html.parser')
    results = sp.select('div.fileGoupBox ul')
    
    for i, result in enumerate(results):
        pdf_date = sp.select('span.date')[i].text[3:]
        
        pdf_temp = requests.get(host + result.select('li')[1].select('a')[0].attrs['href'])

        with open('bok.pdf', 'wb') as f:
            f.write(pdf_temp.content)
        
        parsed = parser.from_file('bok.pdf')
        temp_text = str(parsed['content']).strip()
        temp_df = pd.DataFrame([{'date':pdf_date, 'text':temp_text}])
        result_df = pd.concat([result_df, temp_df])

    if current_page == end_page:
        result_df.to_csv('datas.csv')
        break
    current_page += 1


In [113]:
# 32 페이지 수작업 부분
URL = 'http://www.bok.or.kr/portal/bbs/B0000245/list.do?menuNo=200761&pageIndex=32'
r = requests.get(URL)
sp = BeautifulSoup(r.text, 'html.parser')
results = sp.select('div.fileGoupBox ul')

for i, result in enumerate(results):
    pdf_date = sp.select('span.date')[i].text[3:]

    pdf_temp = requests.get(host + result.select('li')[1].select('a')[0].attrs['href'])

    with open('bok.pdf', 'wb') as f:
        f.write(pdf_temp.content)

    parsed = parser.from_file('bok.pdf')
    temp_text = str(parsed['content']).strip()
    temp_df = pd.DataFrame([{'date':pdf_date, 'text':temp_text}])
    test_df = pd.concat([test_df, temp_df])

In [111]:
URL = 'http://www.bok.or.kr/portal/bbs/B0000245/list.do?menuNo=200761&pageIndex=32'
r = requests.get(URL)
sp = BeautifulSoup(r.text, 'html.parser')
results = sp.select('div.fileGoupBox ul')

In [78]:
results = sp.select('div.fileLink')

In [88]:
results[1].select('a')[0].attrs['href']

'/portal/cmmn/file/fileDown.do?menuNo=200761&atchFileId=KO_00000000000085858&fileSn=1'

In [74]:
pdf_date = sp.select('span.date')[7].text[3:]

In [101]:
for i, result in enumerate(results):
    if i==0:
        continue
    
    pdf_date = sp.select('span.date')[i].text[3:]

    pdf_temp = requests.get(host + results[i].select('a')[0].attrs['href'])

    with open('bok.hwp', 'wb') as f:
        f.write(pdf_temp.content)

    parsed = parser.from_file('bok.hwp')
    temp_text = str(parsed['content']).strip()
    temp_df = pd.DataFrame([{'date':pdf_date, 'text':temp_text}])
    test_df = pd.concat([test_df, temp_df])



In [116]:
test_df.to_csv('bok_data.csv',encoding='utf-8')

In [2]:
import pandas as pd
bok_df = pd.read_csv('bok_datas.csv')

In [3]:
bok_df = bok_df[['date', 'text']]

In [62]:
bok_df.iloc[:10] # 앞부분 다름 index 299 양식 다름 - 여러 개 있음 index 300~ 양식 다름

,date,text
0,2020.07.10,2020년도 제14차\n\n금융통화위원회(정기) 의사록\n\n한 국 은 행\n\n\...
1,2020.06.30,2020년도 제13차\n\n금융통화위원회(정기) 의사록\n\n한 국 은 행\n\n\...
2,2020.06.16,2020년도 제12차\n\n금융통화위원회(정기) 의사록\n\n한 국 은 행\n\n\...
3,2020.06.02,2020년도 제11차\n\n금융통화위원회(정기) 의사록\n\n한 국 은 행\n\n\...
4,2020.05.06,2020년도 제9차\n\n금융통화위원회(임시) 의사록\n\n한 국 은 행\n\n\n...
5,2020.04.28,2020년도 제8차\n\n금융통화위원회(정기) 의사록\n\n한 국 은 행\n\n\n...
6,2020.04.14,2020년도 제7차\n\n금융통화위원회(정기) 의사록\n\n한 국 은 행\n\n\n...
7,2020.03.31,2020년도 제6차\n\n금융통화위원회(임시) 의사록\n\n한 국 은 행\n\n\n...
8,2020.03.31,2020년도 제5차\n\n금융통화위원회(정기) 의사록\n\n한 국 은 행\n\n\n...
9,2020.03.17,2020년도 제4차\n\n금융통화위원회(정기) 의사록\n\n한 국 은 행\n\n\n...


# 의사록 Section 분리

In [112]:
import re
import numpy as np
def section_split(bok_text):
    bok_text = bok_text.replace('\n', ' ')
    s2_ls = []
    s3_ls = []
    section2 = re.finditer('(?<=원 토의내용)((?!심의결과).)*', bok_text)
    for s2 in section2:
        s2_ls.append(s2.group())
    section3 = re.finditer('(?<=심의결과)[^１]*', bok_text)
    for s3 in section3:
        s3_ls.append(s3.group())
    return ' '.join(s2_ls), ' '.join(s3_ls)


split_text = list(map(section_split, bok_df['text']))

TypeError: iteration over a 0-d array

In [117]:
len(split_text)

310

In [118]:
s2_ls = []
s3_ls = []

for doc in split_text:
    s2_ls.append(doc[0])
    s3_ls.append(doc[1])

In [121]:
bok_df['section2'] = s2_ls

In [122]:
bok_df['section3'] = s3_ls

In [124]:
bok_df[['date', 'section2','section3']].to_csv('bok_split_section.csv', encoding='utf-8')

In [125]:
bok_df.isnull().sum()

date        0
text        0
section2    0
section3    0
dtype: int64

In [127]:
bok_df.iloc[30]['section2']

''

In [132]:
bok_df.iloc[30]['text']

'2018년도 제23차\n\n금융통화위원회(정기) 의사록\n\n한 국 은 행\n\n\n\n- 1 -\n\n1. 일 자 2018년 12월 6일(목)\n\n2. 장 소 금융통화위원회 회의실\n\n3. 출석위원 이 주 열 의 장(총재)\n\n이 일 형 위 원\n\n조 동 철 위 원\n\n고 승 범 위 원\n\n신 인 석 위 원\n\n윤 면 식 위 원(부총재)\n\n임 지 원 위 원\n\n4. 결석위원 없 음\n\n5. 참 여 자 장 호 현 감 사 임 형 준 부총재보\n\n허 진 호 부총재보 신 호 순 부총재보\n\n정 규 일 부총재보 서 봉 국 외자운용원장\n\n이 환 석 조사국장 손 욱 경제연구원장\n\n신 운 금융안정국장 박 종 석 통화정책국장\n\n이 상 형 금융시장국장 이 승 헌 국제국장\n\n장 정 석 금융통화위원회실장 김 현 기 공보관\n\n성 광 진 의사팀장\n\n6. 회의경과\n\n<의안 제39호 - 대부업 등의 등록 및 금융이용자 보호에 관한 법률 시행령\n\n제9조제3항제1호의 연체이자율에 관한 규정 폐지규정(안)>\n\n(１) 담당 부총재보가 대부업 등의 등록 및 금융이용자 보호에 관한 법률 시\n\n행령 개정으로 한국은행법 제11조에 따른 금융기관에 대한 연체이자율 상한을\n\n금융위원회에서 정하게 됨에 따라 관련 규정을 폐지하고자 한다는 내용의 제안설\n\n명을 하였음.\n\n(２) 심의결과\n\n원안대로 가결\n\n\n\n- 2 -\n\n의결사항\n\n대부업 등의 등록 및 금융이용자 보호에 관한 법률 시행령 제9조제3항\n\n제1호의 연체이자율에 관한 규정 을 붙임과 같이 폐지한다.\n\n<붙임> 대부업 등의 등록 및 금융이용자 보호에 관한 법률 시행령 제9조\n\n제3항제1호의 연체이자율에 관한 규정 폐지규정(안)'

In [1]:
import pandas as pd
bok_datas = pd.read_csv('bok_split_section.csv',encoding='utf-8')
bok_datas = bok_datas[['date', 'section2', 'section3']]


In [6]:
bok_datas

,date,section2,section3
0,2020.07.10,관련부서는 금번 보고서 작성의 기본방향과 위원협의회에서 논의된 주요 내용 ...,원안대로 가결 의결사항 한국은행법 제96조 제1항에 의거 거시 금융안정상황에...
1,2020.06.30,관련부서는 본 안건과 관련하여 위원협의회 등에서 논의된 내용을 다음과 같 ...,원안대로 가결 의결사항 한국은행법 제96조 제1항에 의거 통화신용정책 수행상...
2,2020.06.16,전일 개최된 동향보고회의에서는 다음과 같은 토의가 있었음. (가) 경제전망(2...,"앞서의 토의결과를 반영하여 다음과 같이 통화정책방향에 대한 의결문을 작 성, ..."
3,2020.06.02,관련부서는 본 안건과 관련하여 5월 11일 위원협의회에서 논의된 내용을 다 음...,의결문 작성·가결 의결사항 2020년 5월 18일부터 적용할 한국은행의 금융...
4,2020.05.06,- 2 - 제안설명에 이어 관련부서에서는 본 안건에 대해 세 차례(4월 7...,원안대로 가결 의결사항 ｢금융안정특별대출제도 신설 및 운용방안｣을 붙임과 같...
...,...,...,...
305,2005.11.08,일부 위원은 수도권 소재 중소기업에 대한 세금 감면폭 축소 등에 관한 일부 ...,를 다음과 같이 보고하 음 - 2 - 「통화신용정책보고서(2005년 9월)...
306,2005.10.25,"｢국내외 경제동향｣과 관련하여, 일부 위원은 7월 전망시보다 높은 수준을...","의결문 작성․가결 (다만, 김태동 위원은 이에 대해 명백히 반대..."
307,2005.09.27,"｢국내외 경제동향｣과 관련하여, 일부 위원은 소비, 수출, 건설투자가 견조...","의결문 작성․가결 (다만, 김태동 위원은 이에 대해 명백히 반대..."
308,2005.08.23,"｢국내외 경제동향｣과 관련하여, 일부 위원은 최근 이란의 새 대통령 취임 ...","- 14 - 의결문 작성․가결 (다만, 김태동 위원은 이에..."


In [14]:
i['section2']

float

In [39]:
for i in bok_datas.iloc:
    if type(i['section2']) != type(np.nan):
        if '정부측' in i['section2']:
            pass
#             print(i['section2'])
            
    else:
        print(i['date'])
        print(i['section3'])

2018.12.26
  원안대로 가결    - 2 -  의결사항  대부업 등의 등록 및 금융이용자 보호에 관한 법률 시행령 제9조제3항  제1호의 연체이자율에 관한 규정 을 붙임과 같이 폐지한다.  <붙임> 대부업 등의 등록 및 금융이용자 보호에 관한 법률 시행령 제9조  제3항제1호의 연체이자율에 관한 규정 폐지규정(안)
2017.02.14
            원안대로 가결           의결사항       ｢한국은행의 금융기관에 대한 여수신이율｣, ｢한국은행의 금융기관대출규정｣ 및 ｢공개시장운영규정｣을 붙임과 같이 개정한다.       <붙임> ｢한국은행의 금융기관에 대한 여수신이율｣ 개정(안)(생략)               ｢한국은행의 금융기관대출규정｣ 개정(안)(생략)               ｢공개시장운영규정｣ 개정(안)(생략)
2012.01.31
( )２ 원안대로 가결  의결사항  한국은행의 금융기관에 대한 여수신이율 을 붙임과 같이 개정한다.｢ ｣ 붙임 한국은행의 금융기관에 대한 여수신이율 개정안 생략< > ( )( )｢ ｣  한국은행 홈페이지 통화정책 금융통화위원회 의결사항 참조( ‘ - - ’ )  의안 제 호 금융기관 예금지급준비규정 개정안< 59 ( )>― 「 」  담당부총재보가 한국은행법 및 동법 시행령 일부 개정에 따라 지급준비( ) ( )２ 원안대로 가결  의결사항  금융기관 예금지급준비규정 을 붙임과 같이 개정한다.｢ ｣ 붙임 금융기관 예금지급준비규정 개정안 생략< > ( )( )｢ ｣  한국은행 홈페이지 통화정책 금융통화위원회 의결사항 참조( ‘ - - ’ )  의안 제 호 외화예금지급준비규정 개정안< 60 ( )>― 「 」  담당부총재보가 한국은행법 개정으로 지급준비금의 계산기간 등이 변경( ) ( )２ 원안대로 가결    - 3 -  의결사항  외화예금지급준비규정 을 붙임과 같이 개정한다.｢ ｣ 붙임 외화예금지급준비규정 개정안 생략< > ( )( )｢ ｣  한국은행 홈페이지 통화정책 금융통화위원회 의결사항 참조( ‘ - 

In [9]:
test_str = bok_datas['section2'][50]



In [10]:
test_str

'   전일 개최된 동향보고회의에서는 다음과 같은 토의가 있었음.   (가) 국내외 경제동향 및 평가   일부 위원은 9∼10월 실물경제 지표를 묶어서 보면 서비스업 생산이 늘어나는 가운데 소매판매도 비교적 양호한 모습을 나타낸 것으로 보인다고 언급하고, 수출 호조에 따른 영향이 점차 소비로 파급되고 있다고 해석할 수 있는지 관련부서의 견해를 물었으며,   이에 대해 관련부서에서는 9∼10월 소매판매의 증가는 수출 호조 이외에 연휴 효과와 정부 정책에도 일정부분 기인한 측면이 있는 데다 최근 들어 고용시장의 여건도 개선세가 다소 주춤한 모습을 보이고 있어 수출 증가에 따른 영향이 본격적으로 소비로 파급되고 있다고 보기는 어려울 것으로 판단된다고 답변하였음. 다만 수출의 높은 증가세가 기업의 수익성 개선, 경제지표의 호전 등을 통해 소비심리에도 긍정적으로 작용하고 있으므로 적어도 간접적으로는 소비에 영향을 미치고 있다고 해석할 수 있다고 덧붙였음.   이어서 동 위원은 중국과의 사드(THAAD) 관련 해빙 분위기 등을 반영하여 내년 가계소득이나 고용시장의 여건을 다소 낙관적으로 전망하고 있는 것은 아닌지 관련부서에 물었으며,   이에 대해 관련부서에서는 최근 한·중 관계의 변화를 고려하면 중국인 관광객 등의 회복속도가 지난 10월 전망시 예상보다 좀 더 빨라질 가능성이 있다고 답변하였음.   또한 동 위원은 최근 관련지표의 흐름에 비추어 앞으로 반도체를 제외한 산업에서도 설비투자가 점점 늘어날 것으로 보이는지 관련부서의 견해를 물었으며,   이에 대해 관련부서에서는 조선 등 일부 업종은 여전히 부진하겠지만 자동차, 화학 등은 신기술 분야를 중심으로 설비투자가 계속 늘어날 것으로 보인다고 답변하였음.   아울러 동 위원은 최근 주요국 통화에 비해 원화의 절상폭이 커지고 있다고 언급하고, 수출의 환율 민감도가 글로벌 분업관계(global value chain) 심화 등으로 과거보다 작아진 가운데 실질실효환율의 절상폭도 아직은 제한적인 것으로 보여 그 영향이 크지는 

In [8]:
from ekonlpy.tag import Mecab

mecab= Mecab()
tokens = mecab.pos(test_str)
mecab.sent_words(tokens)
mecab.replace_synonyms(tokens)
mecab.lemmatize(tokens)


[('관련부서', 'NNG'),
 ('는', 'JX'),
 ('금번', 'NNG'),
 ('보고서', 'NNG'),
 ('작성', 'NNG'),
 ('의', 'JKG'),
 ('기본', 'NNG'),
 ('방향', 'NNG'),
 ('과', 'JC'),
 ('위원', 'NNG'),
 ('협의회', 'NNG'),
 ('에서', 'JKB'),
 ('논의', 'NNG'),
 ('되', 'VV'),
 ('주요', 'NNG'),
 ('내용', 'NNG'),
 ('을', 'JKO'),
 ('다음', 'NNG'),
 ('과', 'JKB'),
 ('같이', 'MAG'),
 ('보', 'VV'),
 ('고', 'EC'),
 ('하', 'VX'),
 ('였', 'EP'),
 ('음', 'ETN'),
 ('.', 'SF'),
 ('먼저', 'MAG'),
 ('금번', 'NNG'),
 ('보고서', 'NNG'),
 ('에서', 'JKB'),
 ('는', 'JX'),
 ('코로나', 'NNG'),
 ('19', 'SN'),
 ('확산', 'NNG'),
 ('에', 'JKB'),
 ('따르', 'VV'),
 ('유례없', 'VA'),
 ('는', 'ETM'),
 ('금융', 'NNG'),
 ('·', 'SC'),
 ('실물', 'NNG'),
 ('충격', 'NNG'),
 ('이', 'JKS'),
 ('금융시스템', 'NNG'),
 ('에', 'JKB'),
 ('미치', 'VV'),
 ('는', 'ETM'),
 ('영향', 'NNG'),
 ('을', 'JKO'),
 ('점검', 'NNG'),
 ('하', 'XSV'),
 ('는', 'ETM'),
 ('한편', 'NNG'),
 (',', 'SC'),
 ('국내외', 'NNG'),
 ('정책', 'NNG'),
 ('대응', 'NNG'),
 ('등', 'NNB'),
 ('의', 'JKG'),
 ('효과', 'NNG'),
 ('를', 'JKO'),
 ('고려', 'NNG'),
 ('하', 'XSV'),
 ('여', 'EC'),
 ('금융안정',

# 의사록 데이터 전처리

In [60]:
import random
bok_datas['section3'][random.randint(0,310)]

' 의결문 작성 가결( ) : ·６  앞서의 토의결과를 반영하여 다음과 같이 통화정책방향에 대한 의결문을  작성 위원 전원 찬성으로 가결,  의결사항  통화정책방향 을 다음과 같이 결정한다.｢ ｣  금융통화위원회는 다음 통화정책방향 결정시까지 한국은행 기준금리□ 를 현 수준 에서 유지하여 통화정책을 운용하기로 하였음(2.00%)  최근 국내 경기는 수출이 확대되고 내수가 꾸준히 증가하는 등 회복□ 세를 지속하고 있으며 앞으로도 이와 같은 추세가 이어질 것으로 전  망됨 다만 일부 국가의 과다채무 문제 등으로 향후 성장경로의 불확.  실성은 상존하고 있는 것으로 판단됨  소비자물가는 상승세가 다소 둔화되었으며 최근의 국제원자재가격 움□ 직임 등에 비추어 당분간 안정된 모습을 보일 것으로 예상됨 부동산.  시장에서는 가격이 완만하게 상승하였으나 거래는 부진하였음  금융시장에서는 일부 유럽국가의 재정위기 등 해외 불안요인이 완화□ 되면서 주가가 반등하고 환율이 하락하였으며 주택담보대출은 계절요  인 등으로 소폭 증가에 그쳤음  앞으로 통화정책은 당분간 금융완화기조를 유지하면서 경기회복세 지□ 속에 도움이 되는 방향으로 운용해 나갈 것임  의안 제 호 년도 연차보고서안11 2009 ( )〈 ― 〉  담당 부총재보가 한국은행법 제 조에 의거 매 회계연도 경과후( ) 102 ( )２  심의위원회에 회부  심의위원회는 심훈 박봉흠 김대식 최도성 강명헌 이주열 위원( • • • • •  인으로 구성되었으며 주관위원으로 심훈 위원이 선정됨6 , )  의안 제 호 통화신용정책 보고서 년 월 안12 (2010 3 )( )〈 ― 〉  담당 부총재보가 한국은행법 제 조 제 항에 의거 통화신용정책( ) 96 1 ( )２  심의위원회에 회부  심의위원회는 심훈 박봉흠 김대식 최도성 강명헌 이주열 위원( • • • • •  인으로 구성되었으며 주관위원으로 최도성 위원이 선정됨6 , )    - 17 -  별 첨( )  통화정책방향 관련 보고내용｢ ｣  국내외 경제동향.Ⅰ  국외경제동향1.

In [65]:
# Section 2

import re

for se2 in bok_datas.iloc:
    if type(se2['section2']) != type(np.nan):
        se2.section2 = se2.section2.strip()
        se2.section2 = re.sub('- [0-9]{1,2} -', '', se2.section2) # 페이지번호 삭제
        se2.section2 = re.sub('\([３４５６７８９0-9 ]*\)', '', se2.section2) # Section 분리기준 삭제
        se2.section2 = re.sub('\([가-힣]*\)', '', se2.section2) #가나다기호 삭제

# '토의결론.*' # 맨 끝 제거
# '관련부서는 ((?!보고하였음).)* 보고하였음\.'
# '일부 위원은'
# se2.replace('  ', '')        

bok_datas

,date,section2,section3
0,2020.07.10,관련부서는 금번 보고서 작성의 기본방향과 위원협의회에서 논의된 주요 내용 ...,원안대로 가결 의결사항 한국은행법 제96조 제1항에 의거 거시 금융안정상황에...
1,2020.06.30,관련부서는 본 안건과 관련하여 위원협의회 등에서 논의된 내용을 다음과 같 ...,원안대로 가결 의결사항 한국은행법 제96조 제1항에 의거 통화신용정책 수행상...
2,2020.06.16,전일 개최된 동향보고회의에서는 다음과 같은 토의가 있었음. 경제전망(2020.5...,"앞서의 토의결과를 반영하여 다음과 같이 통화정책방향에 대한 의결문을 작 성, ..."
3,2020.06.02,관련부서는 본 안건과 관련하여 5월 11일 위원협의회에서 논의된 내용을 다 음과 ...,의결문 작성·가결 의결사항 2020년 5월 18일부터 적용할 한국은행의 금융...
4,2020.05.06,"제안설명에 이어 관련부서에서는 본 안건에 대해 세 차례(4월 7일, 4월 9일, 4...",원안대로 가결 의결사항 ｢금융안정특별대출제도 신설 및 운용방안｣을 붙임과 같...
...,...,...,...
305,2005.11.08,일부 위원은 수도권 소재 중소기업에 대한 세금 감면폭 축소 등에 관한 일부 언론...,를 다음과 같이 보고하 음 - 2 - 「통화신용정책보고서(2005년 9월)...
306,2005.10.25,"｢국내외 경제동향｣과 관련하여, 일부 위원은 7월 전망시보다 높은 수준을 지속하...","의결문 작성․가결 (다만, 김태동 위원은 이에 대해 명백히 반대..."
307,2005.09.27,"｢국내외 경제동향｣과 관련하여, 일부 위원은 소비, 수출, 건설투자가 견조한 증...","의결문 작성․가결 (다만, 김태동 위원은 이에 대해 명백히 반대..."
308,2005.08.23,"｢국내외 경제동향｣과 관련하여, 일부 위원은 최근 이란의 새 대통령 취임 이후 ...","- 14 - 의결문 작성․가결 (다만, 김태동 위원은 이에..."


In [96]:
# Section 3

for se3 in bok_datas.iloc:
    if type(se3['section3']) != type(np.nan):
        se3.section3 = se3.section3.strip()
        se3.section3 = re.sub('\(생략\)', '', se3.section3) # 생략기호 삭제
        se3.section3 = re.sub('\<붙임\>', '', se3.section3) # 붙임기호 삭제
        se3.section3 = re.sub('개정\(안\)', '', se3.section3) # 무의미 단어 삭제
        se3.section3 = re.sub('원안대로 가결', '', se3.section3) # 무의미 단어 삭제
        se3.section3 = re.sub('의결문 작성.가결', '', se3.section3) # 무의미 단어 삭제
        se3.section3 = re.sub('의결사항', '', se3.section3) # 무의미 단어 삭제
        se3.section3 = re.sub('- [0-9]{1,2} -', '', se3.section3) # 페이지번호 삭제
        
# '앞서의 .* 였음.'

bok_datas

,date,section2,section3,doc
0,2020.07.10,관련부서는 금번 보고서 작성의 기본방향과 위원협의회에서 논의된 주요 내용 ...,한국은행법 제96조 제1항에 의거 거시 금융안정상황에 대한 평가보고서를 붙임과 같...,관련부서는 금번 보고서 작성의 기본방향과 위원협의회에서 논의된 주요 내용 ...
1,2020.06.30,관련부서는 본 안건과 관련하여 위원협의회 등에서 논의된 내용을 다음과 같 ...,한국은행법 제96조 제1항에 의거 통화신용정책 수행상황에 대한 보고서를 붙임과 같...,관련부서는 본 안건과 관련하여 위원협의회 등에서 논의된 내용을 다음과 같 ...
2,2020.06.16,전일 개최된 동향보고회의에서는 다음과 같은 토의가 있었음. 경제전망(2020.5...,"앞서의 토의결과를 반영하여 다음과 같이 통화정책방향에 대한 의결문을 작 성, 가결...",전일 개최된 동향보고회의에서는 다음과 같은 토의가 있었음. 경제전망(2020.5...
3,2020.06.02,관련부서는 본 안건과 관련하여 5월 11일 위원협의회에서 논의된 내용을 다 음과 ...,2020년 5월 18일부터 적용할 한국은행의 금융기관에 대한 금융중개지원대출 의 ...,관련부서는 본 안건과 관련하여 5월 11일 위원협의회에서 논의된 내용을 다 음과 ...
4,2020.05.06,"제안설명에 이어 관련부서에서는 본 안건에 대해 세 차례(4월 7일, 4월 9일, 4...",｢금융안정특별대출제도 신설 및 운용방안｣을 붙임과 같이 의결한다. ｢금융안정특별...,"제안설명에 이어 관련부서에서는 본 안건에 대해 세 차례(4월 7일, 4월 9일, 4..."
...,...,...,...,...
305,2005.11.08,일부 위원은 수도권 소재 중소기업에 대한 세금 감면폭 축소 등에 관한 일부 언론...,를 다음과 같이 보고하 음 「통화신용정책보고서(2005년 9월)」는 심의...,일부 위원은 수도권 소재 중소기업에 대한 세금 감면폭 축소 등에 관한 일부 언론...
306,2005.10.25,"｢국내외 경제동향｣과 관련하여, 일부 위원은 7월 전망시보다 높은 수준을 지속하...","(다만, 김태동 위원은 이에 대해 명백히 반대의사를 표시하고 콜금리 목 표 0.2...","｢국내외 경제동향｣과 관련하여, 일부 위원은 7월 전망시보다 높은 수준을 지속하..."
307,2005.09.27,"｢국내외 경제동향｣과 관련하여, 일부 위원은 소비, 수출, 건설투자가 견조한 증...","(다만, 김태동 위원은 이에 대해 명백히 반대의사를 표시하고 콜금리 목 표 0.2...","｢국내외 경제동향｣과 관련하여, 일부 위원은 소비, 수출, 건설투자가 견조한 증..."
308,2005.08.23,"｢국내외 경제동향｣과 관련하여, 일부 위원은 최근 이란의 새 대통령 취임 이후 ...","(다만, 김태동 위원은 이에 대해 명백히 반대의사를 표시하고 ...","｢국내외 경제동향｣과 관련하여, 일부 위원은 최근 이란의 새 대통령 취임 이후 ..."


In [95]:
bok_datas['section3'][308]

'- 14 -            (다만, 김태동 위원은 이에 대해 명백히 반대의사를 표시하고 콜금리 목  표 0.25%p 인상을 주장하 음)            ｢통화정책방향｣을 다음과 같이 결정한다.  □ 실물경제는 수출이 꾸준히 늘어나는 가운데 민간소비가 증가세를 유지  하고 건설투자도 회복기미를 보이고 있으나 설비투자의 개선은 뚜렷하  지 않음            경상수지는 상품수지를 중심으로 흑자기조를 지속하고 있음  □ 물가는 고유가 지속에도 불구하고 수요압력이 미약하여 소비자물가와   근원인플레이션이 모두 안정세를 유지하고 있으나 부동산가격의 높은   오름세가 지속되고 있음  □ 금융시장에서는 유동성사정이 전반적으로 원활하고 금융기관 여신도   주택담보대출을 중심으로 비교적 큰 폭으로 늘어나고 있음  □ 이와 같은 점을 종합적으로 감안하여 다음 통화정책방향 결정시까지   콜금리(무담보 익일물 기준) 목표를 현 수준(연 3.25%)에서 유지하는   방향으로 통화정책을 운용함    - 15 -  (별 첨)  ｢통화정책방향｣ 관련 보고내용  Ⅰ. 국내외 경제동향     1. 해외경제동향          가. 주요국경제  미국경제는 1․4분기 GDP 성장률이 3.8%로 상향 수정(당초 잠정치 3.5%)  된 가운데 5월중 주택경기 호조세가 지속되고 산업생산도 견조한 증가세를 시현하  는 등 견실한 성장세를 지속하 음. 다만 소매판매 및 자본재수주(비국방부문)는   감소하 음  중국경제는 5월중 소매판매 및 수출 호조에 힘입어 산업생산 증가세가 확  대되는 등 고성장을 지속하 음  유로지역 경제는 6월중 유로지역 경기체감지수 및 제조업 구매관리자지수  (PMI)가 기준치를 하회하고 실업률도 높은 수준을 지속하는 등 회복이 지연되는   모습이었음  일본경제는 4월중 증가를 보 던 가계소비, 광공업생산 및 수출이 5월중   감소로 반전하는 등 회복세가 주춤하 음      나. 국제금융시장  미국 주가는 6월 하순 이후 유가급등 등으로 하락 반전하 으나 유로지역

In [97]:
# 합치기

temp = []

for se in bok_datas.iloc:
    
    if type(se.section2) == type(np.nan):
        se.section2 = ''
    if type(se.section3) == type(np.nan):
        se.section3 = ''
        
    temp.append(se['section2'] + se['section3'])
    
bok_datas['doc'] = temp

In [98]:
bok_datas[['date', 'doc']].to_csv('bok_final.csv') # 13117 kb
bok_datas[['date', 'doc']].to_json('bok_final.json') # 24516 kb

In [99]:
pd.read_json('bok_final.json')

,date,doc
0,2020-07-10,관련부서는 금번 보고서 작성의 기본방향과 위원협의회에서 논의된 주요 내용 ...
1,2020-06-30,관련부서는 본 안건과 관련하여 위원협의회 등에서 논의된 내용을 다음과 같 ...
2,2020-06-16,전일 개최된 동향보고회의에서는 다음과 같은 토의가 있었음. 경제전망(2020.5...
3,2020-06-02,관련부서는 본 안건과 관련하여 5월 11일 위원협의회에서 논의된 내용을 다 음과 ...
4,2020-05-06,"제안설명에 이어 관련부서에서는 본 안건에 대해 세 차례(4월 7일, 4월 9일, 4..."
...,...,...
305,2005-11-08,일부 위원은 수도권 소재 중소기업에 대한 세금 감면폭 축소 등에 관한 일부 언론...
306,2005-10-25,"｢국내외 경제동향｣과 관련하여, 일부 위원은 7월 전망시보다 높은 수준을 지속하..."
307,2005-09-27,"｢국내외 경제동향｣과 관련하여, 일부 위원은 소비, 수출, 건설투자가 견조한 증..."
308,2005-08-23,"｢국내외 경제동향｣과 관련하여, 일부 위원은 최근 이란의 새 대통령 취임 이후 ..."
